In [1]:
%matplotlib inline
# %matplotlib qt # Choose %matplotlib qt to plot to an interactive window (note it may show up behind your browser)
# Make some of the relevant imports
import cv2 # OpenCV for perspective transform
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import scipy.misc # For saving images as needed
import glob  # For reading in a list of images from a folder
import imageio
imageio.plugins.ffmpeg.download()

# Import pandas and read in csv file as a dataframe
import pandas as pd
# Change the path below to your data directory
# If you are in a locale (e.g., Europe) that uses ',' as the decimal separator
# change the '.' to ','
df = pd.read_csv('../my_testdata/robot_log.csv', delimiter=';', decimal='.')
csv_img_list = df["Path"].tolist() # Create list of image pathnames
# Read in ground truth map and create a 3-channel image with it
ground_truth = mpimg.imread('../calibration_images/map_bw.png')
ground_truth_3d = np.dstack((ground_truth*0, ground_truth*255, ground_truth*0)).astype(np.float)

# Creating a class to be the data container
# Will read in saved data from csv file and populate this object
# Worldmap is instantiated as 200 x 200 grids corresponding 
# to a 200m x 200m space (same size as the ground truth map: 200 x 200 pixels)
# This encompasses the full range of output position values in x and y from the sim
class Databucket():
    def __init__(self):
        self.images = csv_img_list  
        self.xpos = df["X_Position"].values
        self.ypos = df["Y_Position"].values
        self.yaw = df["Yaw"].values
        self.count = 0 # This will be a running index
        self.worldmap = np.zeros((200, 200, 3)).astype(np.float)
        self.ground_truth = ground_truth_3d # Ground truth worldmap

# Instantiate a Databucket().. this will be a global variable/object
# that you can refer to in the process_image() function below
data = Databucket()


In [2]:
def process_image(img):
    def perspect_transform(img, src, dst):
                M = cv2.getPerspectiveTransform(src, dst)
                warped = cv2.warpPerspective(img, M, (img.shape[1], img.shape[0]))# keep same size as input image
#                 mask = cv2.warpPerspective(np.ones_like(img[:,:,0]), M, (img.shape[1], img.shape[0]))
                return warped

    def color_thresh(img, rgb_thresh=(180, 180, 180)):
                color_select = np.zeros_like(img[:,:,0])
                above_thresh = (img[:,:,0] > rgb_thresh[0]) \
                            & (img[:,:,1] > rgb_thresh[1]) \
                            & (img[:,:,2] > rgb_thresh[2])
                color_select[above_thresh] = 1
                return color_select

    def obs(img, rgb_thresh=(120, 120, 100)):
                obs_select = np.zeros_like(img[:,:,0])
                below_thresh = (img[:,:,0] < rgb_thresh[0]) \
                            & (img[:,:,1] < rgb_thresh[1]) \
                            & (img[:,:,2] < rgb_thresh[2])
                obs_select[below_thresh] = 1
                return obs_select

    def rover_coords(binary_img):
                ypos, xpos = binary_img.nonzero()
                x_pixel = 0.5*(-(ypos - binary_img.shape[0]).astype(np.float))
                y_pixel = 0.5*(-(xpos - binary_img.shape[1]/2 ).astype(np.float))
                return x_pixel, y_pixel

    def to_polar_coords(x_pixel, y_pixel):
                dist = np.sqrt(x_pixel**2 + y_pixel**2)
                angles = np.arctan2(y_pixel, x_pixel)
                return dist, angles

    def rotate_pix(xpix, ypix, yaw):
                yaw_rad = yaw * np.pi / 180
                xpix_rotated = (xpix * np.cos(yaw_rad)) - (ypix * np.sin(yaw_rad))
                ypix_rotated = (xpix * np.sin(yaw_rad)) + (ypix * np.cos(yaw_rad))
                return xpix_rotated, ypix_rotated

    def translate_pix(xpix_rot, ypix_rot, xpos, ypos, scale): 
                xpix_translated = (xpix_rot / scale) + xpos
                ypix_translated = (ypix_rot / scale) + ypos
                return xpix_translated, ypix_translated

    def pix_to_world(xpix, ypix, xpos, ypos, yaw, world_size, scale):
                xpix_rot, ypix_rot = rotate_pix(xpix, ypix, yaw)
                xpix_tran, ypix_tran = translate_pix(xpix_rot, ypix_rot, xpos, ypos, scale)
                x_pix_world = np.clip(np.int_(xpix_tran), 0, world_size - 1)
                y_pix_world = np.clip(np.int_(ypix_tran), 0, world_size - 1)
                return x_pix_world, y_pix_world
    
    dst_size = 5 
    bottom_offset = 6
    source = np.float32([[14, 140], [301 ,140],[200, 96], [118, 96]])
    destination = np.float32([[img.shape[1]/2 - dst_size, img.shape[0] - bottom_offset],
                          [img.shape[1]/2 + dst_size, img.shape[0] - bottom_offset],
                          [img.shape[1]/2 + dst_size, img.shape[0] - 2*dst_size - bottom_offset], 
                          [img.shape[1]/2 - dst_size, img.shape[0] - 2*dst_size - bottom_offset],
                          ])  

 
    threshed = color_thresh(img)
#     threshedobs = np.absolute(np.float32(threshed) - 1)
    threshedobs = obs(img)
    warped = perspect_transform(threshed, source, destination)
    warped1 = perspect_transform(threshedobs, source, destination)

    rock_img = mpimg.imread('../calibration_images/example_rock1.jpg')
    gold = np.uint8([[[140,110,0]]])
    hsv_gold = cv2.cvtColor(gold,cv2.COLOR_BGR2HSV)
    hue = hsv_gold[0][0][0]
    hsv = cv2.cvtColor(rock_img, cv2.COLOR_BGR2HSV)
    lower_gold = np.array([hue-10,100,100])
    upper_gold = np.array([hue+10,255,255])
    gmask = cv2.inRange(hsv, lower_gold, upper_gold)
    res = cv2.bitwise_and(rock_img, rock_img, mask= gmask)
    
# Calculate pixel values in rover-centric coords and distance/angle to all pixels
#     xpix, ypix = rover_coords(threshed)
#     xobs, yobs = rover_coords(threshedobs)
    xpix, ypix = rover_coords(warped)
    xobs, yobs = rover_coords(warped1)
    xgol, ygol = rover_coords(gmask)
    dist, angles = to_polar_coords(xpix, ypix)
    mean_dir = np.mean(angles)

# Get navigable pixel positions in world coords
    scale = 2*dst_size
    xpos = data.xpos[data.count]
    ypos = data.ypos[data.count]
    yaw = data.yaw[data.count]
    world_size = data.worldmap.shape[0]
    
    x_world, y_world = pix_to_world(xpix, ypix, xpos, ypos, yaw, world_size, scale)
    data.worldmap[y_world, x_world, 2] += 50
#     data.worldmap[y_world, x_world, 2] = 255
# Get obstruction pixel positions in world coords
    xo_world, yo_world = pix_to_world(xobs, yobs, xpos, ypos, yaw, world_size, scale)
    data.worldmap[yo_world, xo_world, 0] += 100
#     data.worldmap[yo_world, xo_world, 0] = 255

# Get gold pixel positions in world coords  
    xgw, ygw = pix_to_world(xgol, ygol, xpos, ypos, yaw, world_size, scale)
    data.worldmap[ygw, xgw, :] = 255
    
    nav_pix = data.worldmap[:,:,2] > 0
    data.worldmap[nav_pix, 0] = 0

    # 7) Make a mosaic image, below is some example code
        # First create a blank image (can be whatever shape you like)
    output_image = np.zeros((img.shape[0] + data.worldmap.shape[0], img.shape[1]*2, 3))
        # Next you can populate regions of the image with various output
        # Here I'm putting the original image in the upper left hand corner
    output_image[0:img.shape[0], 0:img.shape[1]] = img

        # Let's create more images to add to the mosaic, first a warped image
        # Add the warped image in the upper right hand corner
    warped = perspect_transform(img, source, destination)
    output_image[0:img.shape[0], img.shape[1]:] = warped
       # Overlay worldmap with ground truth map
    map_add = cv2.addWeighted(data.worldmap, 1, data.ground_truth, 0.5, 0)
       # Flip map overlay so y-axis points upward and add to output_image 
    output_image[img.shape[0]:, 0:data.worldmap.shape[1]] = np.flipud(map_add)

       # Then putting some text over the image
    cv2.putText(output_image,"Populate this image with your analyses to make a video!", (20, 20), 
                cv2.FONT_HERSHEY_COMPLEX, 0.4, (255, 255, 255), 1)
    if data.count < len(data.images) - 1:
        data.count += 1 # Keep track of the index in the Databucket()
  
    return output_image     


In [3]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from moviepy.editor import ImageSequenceClip


# Define pathname to save the output video
output = '../output/test_mapping.mp4'
data = Databucket() # Re-initialize data in case you're running this cell multiple times
clip = ImageSequenceClip(data.images, fps=60) # Note: output video will be sped up because 
                                          # recording rate in simulator is fps=25
new_clip = clip.fl_image(process_image) #NOTE: this function expects color images!!
%time new_clip.write_videofile(output, audio=False)

[MoviePy] >>>> Building video ../output/test_mapping.mp4
[MoviePy] Writing video ../output/test_mapping.mp4


100%|██████████| 419/419 [00:07<00:00, 54.16it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ../output/test_mapping.mp4 

CPU times: user 6.61 s, sys: 192 ms, total: 6.8 s
Wall time: 7.99 s


In [4]:
from IPython.display import HTML
import io
import base64
video = io.open(output, 'r+b').read()
encoded_video = base64.b64encode(video)
HTML(data='''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded_video.decode('ascii')))

In [ ]:

from IPython.display import HTML
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(output))

### Below is an alternative way to create a video in case the above cell did not work.